<a href="https://colab.research.google.com/github/Teasotea/BioNER-and-RD/blob/main/Scibert_TokenClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [3]:
import os
import math
import random
import csv
import sys

import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import statistics as stats

In [133]:
import nltk
from nltk.tokenize import TreebankWordTokenizer as twt
import spacy
from spacy import displacy

In [4]:
!nvidia-smi

Fri Jun 24 22:21:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import torch
print('pytorch version:', torch.__version__)
print('GPU:',torch.cuda.get_device_name(0))

pytorch version: 1.11.0+cu113
GPU: Tesla T4


# Load Data

In [7]:
with open('/content/data0.txt') as f:
      eval_article = f.read()

In [ ]:
# train_df = pd.read_csv('/content/cdr_dner_train_df.csv')
# test_df = pd.read_csv('/content/cdr_dner_test_df.csv')
# dev_df = pd.read_csv('/content/cdr_dner_dev_df.csv')

In [ ]:
# CID_train_df = train_df[train_df['xloc'] == "CID"].reset_index(drop=True)
# DNER_train_df = train_df[train_df['xloc'] != "CID"].reset_index(drop=True)

# CID_test_df = test_df[test_df['xloc'] == "CID"].reset_index(drop=True)
# DNER_test_df = test_df[test_df['xloc'] != "CID"].reset_index(drop=True)

# CID_dev_df = dev_df[dev_df['xloc'] == "CID"].reset_index(drop=True)
# DNER_dev_df = dev_df[dev_df['xloc'] != "CID"].reset_index(drop=True)

In [84]:
iob_train = pd.read_csv('/content/iob_train.csv',converters={"tokens": lambda x: x.strip("[]").replace("'","").split(", "), "iob_tags": lambda x: x.strip("[]").replace("'","").split(", ")})
iob_test = pd.read_csv('/content/iob_test.csv',converters={"tokens": lambda x: x.strip("[]").replace("'","").split(", "), "iob_tags": lambda x: x.strip("[]").replace("'","").split(", ")})
iob_dev = pd.read_csv('/content/iob_dev.csv',converters={"tokens": lambda x: x.strip("[]").replace("'","").split(", "), "iob_tags": lambda x: x.strip("[]").replace("'","").split(", ")})

In [85]:
iob_dev

,tokens,iob_tags
0,"[Tricuspid, valve, regurgitation, and, lithium...","[B-Disease, I-Disease, I-Disease, O, B-Chemica..."
1,"[Phenobarbital-induced, dyskinesia, in, a, neu...","[B-Chemical, B-Disease, O, O, B-Disease, O, O,..."
2,"[Acute, changes, of, blood, ammonia, may, pred...","[O, O, O, O, B-Chemical, O, O, O, O, O, O, B-C..."
3,"[Effects, of, calcitonin, on, rat, extrapyrami...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[Development, of, isoproterenol-induced, cardi...","[O, O, B-Chemical, B-Disease, O, O, O, O, B-Di..."
...,...,...
487,"[Reversible, cerebral, lesions, associated, wi...","[O, B-Disease, I-Disease, O, O, B-Chemical, O,..."
488,"[Antagonism, of, diazepam-induced, sedative, e...","[O, O, B-Chemical, O, O, O, B-Chemical, O, O, ..."
489,"[Enhanced, stimulus-induced, neurotransmitter,...","[O, O, O, O, O, B-Chemical, B-Disease, O, O, O..."
490,"[Ocular, manifestations, of, juvenile, rheumat...","[O, O, O, B-Disease, I-Disease, O, O, O, O, O,..."


# Importing BERT-sklearn

In [57]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .
import os
os.chdir("bert-sklearn")
print(os.listdir())

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 8.19 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/bert-sklearn/bert-sklearn
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for bert-sklearn: filename=bert_sklearn-0.3.1-py3-none-any.whl size=54247 sha256=9cb128daa35753c2b5edcc58c473f588f31343bd1a849bac8d1ddfa93be2c382
  Stored in directory: /root/.cache/pip/wheels/d8/81/d8

In [58]:
from bert_sklearn import BertClassifier
from bert_sklearn import BertRegressor
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model

In [86]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [91]:
train = iob_train.append(iob_dev, ignore_index=True)
train = train.reset_index(drop = True)
print("Train and dev data: %d sentences, %d tokens"%(len(train), len(flatten(train.tokens.tolist()))))
print("Test data: %d sentences, %d tokens"%(len(iob_test), len(flatten(iob_test.tokens))))

Train and dev data: 981 sentences, 203599 tokens
Test data: 497 sentences, 107867 tokens


In [92]:
X_train, y_train = train.tokens, train.iob_tags
X_test, y_test = iob_test.tokens, iob_test.iob_tags

In [93]:
y_train

0      [B-Chemical, O, O, O, O, O, B-Chemical, O, O, ...
1      [B-Chemical, B-Disease, O, O, O, O, O, O, O, O...
2      [B-Chemical, O, O, O, O, B-Disease, O, O, O, B...
3      [B-Chemical, I-Chemical, O, O, O, O, O, O, O, ...
4      [O, O, O, O, O, O, O, O, B-Chemical, B-Disease...
                             ...                        
976    [O, B-Disease, I-Disease, O, O, B-Chemical, O,...
977    [O, O, B-Chemical, O, O, O, B-Chemical, O, O, ...
978    [O, O, O, O, O, B-Chemical, B-Disease, O, O, O...
979    [O, O, O, B-Disease, I-Disease, O, O, O, O, O,...
980    [B-Disease, I-Disease, O, O, B-Chemical, O, O,...
Name: iob_tags, Length: 981, dtype: object

In [94]:
label_list = list(np.unique(flatten(y_train)))
print("\nNER tags:", label_list)


NER tags: ['B-Chemical', 'B-Disease', 'I-Chemical', 'I-Disease', 'O']


In [95]:
train.head()

,tokens,iob_tags
0,"[Naloxone, reverses, the, antihypertensive, ef...","[B-Chemical, O, O, O, O, O, B-Chemical, O, O, ..."
1,"[Lidocaine-induced, cardiac, asystole., Intrav...","[B-Chemical, B-Disease, O, O, O, O, O, O, O, O..."
2,"[Suxamethonium, infusion, rate, and, observed,...","[B-Chemical, O, O, O, O, B-Disease, O, O, O, B..."
3,"[Galanthamine, hydrobromide, ,, a, longer, act...","[B-Chemical, I-Chemical, O, O, O, O, O, O, O, ..."
4,"[Effects, of, uninephrectomy, and, high, prote...","[O, O, O, O, O, O, O, O, B-Chemical, B-Disease..."


### Finetune SciBERT

1) Use `'scibert-scivocab-cased'` model. [GitHub](https://github.com/allenai/scibert), [paper](https://arxiv.org/pdf/1903.10676.pdf)

2) Set the **`ignore_label`** =`['O']`. Exclude the non named entities.

3) Set the **` max_seq_len`** = 256 - high enough to cover lengths of the token lists

In [96]:
%%time
scibert_model = BertTokenClassifier('scibert-scivocab-cased',
                            max_seq_length=64,
                            epochs=3,
                            gradient_accumulation_steps=2,
                            learning_rate=3e-5,
                            train_batch_size=16,
                            eval_batch_size=16,
                            validation_fraction=0.,                            
                            label_list=label_list,
                            ignore_label=['O'])



scibert_model.fit(X_train, y_train)

Building sklearn token classifier...


100%|██████████| 410521600/410521600 [00:13<00:00, 30429885.83B/s]


Loading scibert-scivocab-cased model...


100%|██████████| 410521600/410521600 [00:13<00:00, 29420235.65B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 981, validation data size: 0



...

Training  :   0%|          | 0/123 [00:00<?, ?it/s]

/content/bert-sklearn/bert_sklearn/model/pytorch_pretrained/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Training  :   0%|          | 0/123 [00:00<?, ?it/s]

Training  :   0%|          | 0/123 [00:00<?, ?it/s]

CPU times: user 2min 7s, sys: 56.1 s, total: 3min 3s
Wall time: 3min 53s


In [97]:
scibert_f1_test = scibert_model.score(X_test, y_test,'macro')
print("Test f1: %0.02f"%(scibert_f1_test))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/32 [00:00<?, ?it/s]

Test f1: 76.60


In [98]:
scibert_y_preds = scibert_model.predict(X_test)
print(classification_report(flatten(y_test), flatten(scibert_y_preds)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/32 [00:00<?, ?it/s]

              precision    recall  f1-score   support

  B-Chemical       0.95      0.75      0.84      5240
   B-Disease       0.85      0.71      0.77      4337
  I-Chemical       0.78      0.69      0.74       530
   I-Disease       0.75      0.69      0.72      2142
           O       0.97      0.99      0.98     95618

    accuracy                           0.96    107867
   macro avg       0.86      0.77      0.81    107867
weighted avg       0.96      0.96      0.96    107867



In [99]:
iter_zip = zip(flatten(X_test),flatten(y_test),flatten(scibert_y_preds))
preds = [" ".join([token, y, y_pred]) for token, y, y_pred in iter_zip]
with open("preds.txt",'w') as f:
    for x in preds:
        f.write(str(x)+'\n') 

!perl ./other_examples/conlleval.pl < preds.txt
!rm preds.txt

processed 107867 tokens with 9582 phrases; found: 7909 phrases; correct: 6769.
accuracy:  95.86%; precision:  85.59%; recall:  70.64%; FB1:  77.40
         Chemical: precision:  92.67%; recall:  73.83%; FB1:  82.19  4177
          Disease: precision:  77.65%; recall:  66.79%; FB1:  71.81  3732


In [102]:
num = 28
tokens = X_test[num]
labels = y_test[num]
scibert_preds  = scibert_y_preds[num]

In [103]:
data = {"token": tokens,"label": labels, "scibert": scibert_preds}
df=pd.DataFrame(data=data)
df.head(50)

,token,label,scibert
0,Syndrome,B-Disease,B-Disease
1,of,I-Disease,I-Disease
2,inappropriate,I-Disease,I-Disease
3,antidiuretic,I-Disease,I-Disease
4,hormone,I-Disease,I-Disease
5,secretion,O,I-Disease
6,associated,O,O
7,with,O,O
8,desvenlafaxine.,B-Chemical,B-Chemical
9,OBJECTIVE,O,O


### Hyperparameter Tuning

In [110]:
# from sklearn.model_selection import GridSearchCV

# params = {'epochs':[3, 4], 'learning_rate':[2e-5, 3e-5, 5e-5]}

# scibert_clf = GridSearchCV(BertTokenClassifier('scibert-scivocab-cased',
#                             max_seq_length=256,
#                             gradient_accumulation_steps=2,
#                             train_batch_size=16,
#                             eval_batch_size=16,
#                             validation_fraction=0.,                            
#                             label_list=label_list,
#                             ignore_label=['O']), 
#                    params,
#                    verbose=True)

# scibert_clf.fit(X_train ,y_train)

# Results of DNER task with SciBERT

In [167]:
eval_data = pd.Series(eval_article.split('\n'))
tokenized_eval_data = eval_data.apply(lambda x:list(twt().tokenize(x)))
tokenized_eval_data

0      [Transductional, analysis, was, applied, to, t...
1      [Bacteriocins, produced, by, six, strains, of,...
2      [The, gamma, haemolysin, of, Staphylococcus, a...
3      [Arginine, and, methionine, transport, by, Asp...
4      [Aphids, transmitted, poly-L-ornithine, (, PLO...
                             ...                        
96     [Most-probable-number, (, MPN, ), and, membran...
97     [The, relationship, between, muscle, surface, ...
98     [The, specificity, of, Ca++, for, the, interac...
99     [The, action, of, a, number, of, compounds, ab...
100                                                   []
Length: 101, dtype: object

In [168]:
eval_predicts_scibert = scibert_model.predict(tokenized_eval_data)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/7 [00:00<?, ?it/s]

In [195]:
print(eval_predicts_scibert[4])

['O', 'O', 'B-Chemical', 'O', 'B-Chemical', 'O', 'O', 'B-Disease', 'I-Disease', 'I-Disease', 'O', 'B-Disease', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'B-Disease', 'O', 'O', 'O', 'B-Disease', 'I-Disease', 'O', 'B-Chemical', 'B-Disease', 'O', 'O', 'O', 'O'

In [177]:
print(len(eval_predicts_scibert), len(eval_data.tolist()))

101 101


In [179]:
def locate_entities(text, annots):
  loc_list = []
  span_list = list(twt().span_tokenize(text))
  for annotation, loc in zip(annots, span_list):
    if annotation != 'O':
      loc = (loc[0], loc[1], str(annotation))
      loc_list.append(loc)
  return loc_list

In [198]:
eval_data_entities = [locate_entities(x,y) for x, y in zip(eval_data.tolist(), eval_predicts_scibert)]
eval_data_entities[99]

[(55, 61, 'B-Chemical'),
 (151, 156, 'B-Chemical'),
 (167, 178, 'B-Chemical'),
 (180, 189, 'B-Chemical'),
 (191, 198, 'B-Chemical'),
 (200, 231, 'B-Chemical'),
 (233, 246, 'B-Chemical'),
 (248, 254, 'B-Chemical'),
 (255, 272, 'I-Chemical'),
 (277, 283, 'B-Chemical'),
 (284, 307, 'B-Chemical'),
 (308, 312, 'I-Chemical'),
 (467, 478, 'B-Chemical'),
 (483, 496, 'B-Chemical'),
 (554, 560, 'B-Chemical'),
 (604, 625, 'B-Chemical'),
 (675, 697, 'B-Chemical'),
 (699, 706, 'B-Chemical'),
 (708, 717, 'B-Chemical'),
 (722, 727, 'B-Chemical'),
 (769, 774, 'B-Chemical'),
 (912, 919, 'B-Chemical'),
 (1072, 1087, 'B-Chemical')]

In [225]:
def display_entities(eval_data, num):
  if len(eval_data_entities[num]) == 0:
    return eval_data
  scibert_annot_data = [(eval_data.tolist()[num],
  {'entities': eval_data_entities[num]})]
  options = {"ents": ['B-Chemical', 'B-Disease','I-Chemical', 'I-Disease'], "colors": {'B-Chemical': "#85C1E9", "B-Disease": "#ff6961", 'I-Chemical': "#85C1E9", "I-Disease": "#ff6961"}}
  nlp = spacy.blank('en')
  raw_text = scibert_annot_data[0][0]
  doc = nlp.make_doc(raw_text)
  spans = scibert_annot_data[0][1]["entities"]
  ents = []
  for span_start, span_end, label in spans:
    ent = doc.char_span(span_start, span_end, label=label)
    if ent is None:
        continue
    ents.append(ent)
  doc.ents = ents
  displacy.render(doc, style="ent",options=options, jupyter=True)

In [207]:
display_entities(eval_data, 99)

In [226]:
for i in range(5,15):
  display_entities(eval_data, i)